In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### diagnosis :癌症的类型；radius：半径，texture：组织结构，perimeter：周长，area：特定区域，smoothness：平滑性，compactness：紧密性；concavity：凹面；concave points：凹点；symmetry：对称；fractal_dimension:分形维度

### 导入数据
 - 发现数据id和Unamed：32没有作用
 - 使用drop命令删除

In [ ]:
df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df = df.drop('id',axis=1)
df

In [ ]:
df = df.drop('Unnamed: 32',axis=1)
df

### 观察诊断，发现癌症的类型只有2种
 - 定义：M种类的癌症=1，B种类的癌症=0

In [ ]:
df.diagnosis.unique()

In [ ]:
df['diagnosis'] = df['diagnosis'].map({'M':1,'B':0})
df.head()

In [ ]:
df.describe()

- 绘制热力图，查看样本中元素的相关性
- 例如：radius_mean和diagnosis之间的相关系数为0.73，属于比较高
- 例如：smoothness和diagnosis之间的相关系数为0.36，属于比较低

In [ ]:
plt.figure(figsize=(20,20))
p = sns.heatmap(df.corr(),annot=True, square=True)
plt.show()

### 查看癌症类型的分布并用直方图显示

In [ ]:
print(df.diagnosis.value_counts())
p = df.diagnosis.value_counts().plot(kind='bar')
plt.legend(labels=['B=0,M=1'],loc='best')
plt.show()

### 获取训练集=特征，标签=诊断

In [ ]:
x_data = df.drop(['diagnosis'],axis=1)
y_data = df['diagnosis']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.3,stratify=y_data)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier

classifiers = [
    KNeighborsClassifier(3),
    LogisticRegression(),
    MLPClassifier(hidden_layer_sizes=(20,50),max_iter=10000),
    RandomForestClassifier(max_depth=9,min_samples_split=3),
    AdaBoostClassifier(),
    BaggingClassifier(),
    ]

log = []
for clf in classifiers:
    clf.fit(x_train,y_train)
    name = clf.__class__.__name__
    
    print('='*30)
    print(name)
    
    print('****Results****')
    test_predictions = clf.predict(x_test)
    acc = accuracy_score(y_test, test_predictions)
    print('Accuracy:{:.4%}'.format(acc))
    log.append([name,acc*100])

print('='*30)
    

In [ ]:
log = pd.DataFrame(log)
log

In [ ]:
log.rename(columns={0:'Classifier',1:'Accuracy'},inplace=True)
log

In [ ]:
sns.barplot(x='Accuracy',y='Classifier',data=log,color='r')

plt.xlabel('Accuracy %')
plt.title('Classifier Accuracy')
plt.show()
print('The best model is :',log.max())